In [196]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from datetime import datetime

## Import Data

In [197]:
# retrieve a list of filenames in the 'q2_data' ending with the extension '.pqt'.
parquet_files = [file for file in os.listdir('q2_data') if file.endswith('.pqt')]
parquet_files

['q2_acctDF_HOLDOUT_final.pqt',
 'q2_outflows_HOLDOUT_2ndhalf_final.pqt',
 'q2_inflows_HOLDOUT_final.pqt',
 'q2_outflows_2ndhalf_final.pqt',
 'q2_consDF_final.pqt',
 'q2_consDF_HOLDOUT_notags_final.pqt',
 'q2_acctDF_final.pqt',
 'q2_inflows_final.pqt',
 'q2_outflows_HOLDOUT_1sthalf_final.pqt',
 'q2_outflows_1sthalf_final.pqt']

In [198]:
# reads multiple pqt files into a dictionary by pandas

df = {} #initializes an empty dictionary

for i in parquet_files:
    
    path = 'q2_data/' + i
    dataset_name = str(i[:-4])
    dataset = pd.read_parquet(path)
    df[dataset_name] = dataset

In [199]:
df['q2_consDF_final'].head()

,prism_consumer_id,evaluation_date,APPROVED,FPF_TARGET
0,658,2022-06-24,1,0.0
1,539,2023-05-10,1,0.0
2,540,2021-12-21,1,0.0
3,787,2022-06-22,1,0.0
4,1141,2022-07-08,1,0.0


In [200]:
df['q2_inflows_final'].head()

,prism_consumer_id,prism_account_id,memo_clean,amount,posted_date,category_description
0,0,acc_0,TRANSFER FROM CHK XXXXXXXXX,25.00,2022-05-04,SELF_TRANSFER
1,0,acc_0,TRANSFER FROM CHK XXXXXXXXX,25.00,2023-01-18,SELF_TRANSFER
2,0,acc_0,TRANSFER FROM CHK XXXXXXXXX,25.00,2023-03-01,SELF_TRANSFER
3,0,acc_0,INTEREST PAYMENT,0.05,2023-02-28,INVESTMENT_INCOME
4,0,acc_0,INTEREST PAYMENT,0.07,2023-01-31,INVESTMENT_INCOME


## Create Features

In [202]:
def consumer_data(consumer_id, end_date):
    # Get the target variable(Y) for the specified consumer ID
    y = df['q2_consDF_final'][df['q2_consDF_final']['prism_consumer_id']==consumer_id].reset_index()['FPF_TARGET'][0]
    
    # Get account information for the specified consumer ID
    account_info = df['q2_acctDF_final'][df['q2_acctDF_final']['prism_consumer_id']== consumer_id]
    
    # Get inflow data for the specified consumer ID up to the end date
    inflows = df['q2_inflows_final'][(df['q2_inflows_final']['prism_consumer_id']== consumer_id)&(df['q2_inflows_final']['posted_date']<= end_date)].sort_values('posted_date')
    
    # Concatenate first and second outflows
    outflows = pd.concat([df['q2_outflows_1sthalf_final'],df['q2_outflows_2ndhalf_final']])
    # Get outflows data for the specified consumer ID up to the end date
    outflows = outflows[(outflows['prism_consumer_id']==consumer_id)&(outflows['posted_date']<= end_date)].sort_values('posted_date')
    
    # Add a 'month' column to the inflow and outflows data
    inflows['month'] = [date.strftime("%Y-%m") for date in inflows['posted_date']]
    outflows['month'] = [date.strftime("%Y-%m") for date in outflows['posted_date']]
    
    return (y,account_info,inflows,outflows)

In [203]:
def positive_remaining_ratio(inflow_df, outflow_df):
    # Group and sum inflow amounts by month
    inflow_sum = inflow_df.groupby('month')['amount'].sum().reset_index()
    
    # Group and sum outflow amounts by month
    outflow_sum = outflow_df.groupby('month')['amount'].sum().reset_index()
    
    # Merge inflow and outflow sums on the 'month' column
    merged_df = inflow_sum.merge(outflow_sum, how='left', on='month')
    
    # Calculate the remaining amount for each month
    merged_df['remaining'] = merged_df['amount_x'] - merged_df['amount_y']
    
    # Determine if the remaining amount is positive for each month
    positive_remaining = (merged_df['remaining'] > 0).astype(int)
    
    # Calculate the ratio of months with positive remaining amounts
    positive_remaining_ratio = positive_remaining.mean()
    
    return positive_remaining_ratio

In [204]:
def credit_ratio(outflow_df):
    # Group by 'month' and 'category_description' and count occurrences
    grouped_counts = outflow_df.groupby(['month', 'category_description']).size().reset_index(name='count')
    
    # Add a 'cumulative_count' column representing the cumulative count for each category
    grouped_counts['cumulative_count'] = grouped_counts.groupby('category_description').cumcount() + 1
    
    # Extract the cumulative count for 'CREDIT_CARD_PAYMENT', or set to 0 if not present
    num_credit = grouped_counts[grouped_counts['category_description'] == 'CREDIT_CARD_PAYMENT']['cumulative_count'].tolist()
    num_credit = num_credit[-1] if num_credit else 0
    
    # Extract the cumulative count for 'LOAN', or set to 0 if not present
    num_loan = grouped_counts[grouped_counts['category_description'] == 'LOAN']['cumulative_count'].tolist()
    num_loan = num_loan[-1] if num_loan else 0
    
    # Calculate the total number of unique months
    num_month = len(set(grouped_counts['month']))
    
    # Calculate the maximum credit monthly ratio, otherwise return 0
    return max(num_credit / num_month, num_loan / num_month) if num_month > 0 else 0


In [205]:
def regular_income_ratio(inflow_df):
    # Group by 'month' and 'category_description' and count occurrences
    grouped_counts = inflow_df.groupby(['month', 'category_description']).size().reset_index(name='count')
    
    # Add a 'cumulative_count' column representing the cumulative count for each category
    grouped_counts['cumulative_count'] = grouped_counts.groupby('category_description').cumcount() + 1
    
    # Extract the cumulative count for 'PAYCHECK', or set to 0 if not present
    num_paycheck = grouped_counts[grouped_counts['category_description'] == 'PAYCHECK']['cumulative_count'].tolist()
    num_paycheck = num_paycheck[-1] if num_paycheck else 0
    
    # Extract the cumulative count for 'PAYCHECK_PLACEHOLDER', or set to 0 if not present
    num_paycheck_placeholder = grouped_counts[grouped_counts['category_description'] == 'PAYCHECK_PLACEHOLDER']['cumulative_count'].tolist()
    num_paycheck_placeholder = num_paycheck_placeholder[-1] if num_paycheck_placeholder else 0
    
    # Calculate the total number of unique months in the income data
    num_month = len(set(inflow_df['month']))
    
    # Calculate the maximum monthly ratio of income; otherwise, return 0
    return max(num_paycheck_placeholder / num_month, num_paycheck / num_month) if num_month > 0 else 0


In [232]:
test_df = df['q2_consDF_final'].sample(n=1000)
test_df.head()

,prism_consumer_id,evaluation_date,APPROVED,FPF_TARGET
776,191,2022-10-07,1,0.0
60,123,2021-07-13,1,0.0
465,45,2021-12-21,1,0.0
2196,5367,2022-09-05,1,0.0
190,23,2022-05-31,1,0.0


In [233]:
pos_remain_ratios = []
credit_ratios = []
income_ratios = []

for i, row in test_data.iterrows():
    consumer_id, end_date = row['prism_consumer_id'], row['evaluation_date']
    y_, c_, inflow_data, outflow_data = consumer_data(consumer_id, end_date)
    
    # Calculate ratios and append to respective lists
    pos_remain_ratio = positive_remaining_ratio(inflow_data, outflow_data)
    pos_remain_ratios.append(pos_remain_ratio)
    
    credit_ratio_val = credit_ratio(outflow_data)
    credit_ratios.append(credit_ratio_val)
    
    income_ratio = regular_income_ratio(inflow_data)
    income_ratios.append(income_ratio)
    
#     print(credit_ratio_val)

In [234]:
test_df['regular_income_ratios'] = income_ratios
test_df['credit_ratios'] = credit_ratios
test_df['pos_remain_ratios'] = pos_remain_ratios

In [235]:
test_df.head()

,prism_consumer_id,evaluation_date,APPROVED,FPF_TARGET,regular_income_ratios,credit_ratios,pos_remain_ratios
776,191,2022-10-07,1,0.0,1.000000,0.857143,0.571429
60,123,2021-07-13,1,0.0,1.000000,0.880000,0.480000
465,45,2021-12-21,1,0.0,0.473684,0.947368,0.368421
2196,5367,2022-09-05,1,0.0,1.000000,0.833333,0.333333
190,23,2022-05-31,1,0.0,0.923077,1.000000,0.692308


In [236]:
test_df.shape 

(1000, 7)

Regular Income Ratios: Check whether each customer receives a regular 'PAYCHECK' each month. Calculate the occurrence of 'PAYCHECK' for each month and determine the ratio to assess regularity.

Credit Ratios: Examine whether customers consistently repay their credit or loans by analyzing historical credit payments. Calculate the occurrences of 'CREDIT_CARD_PAYMENT' and 'LOAN' each month and determine the ratios to gauge regularity.

Positive Remain Ratios: Assess whether each customer maintains a positive cash flow. Calculate monthly inflows and outflows, then use the remaining balance to determine the cash flow.```




## Model to Predict

In [237]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming your data is stored in a DataFrame called 'df'
features = test_df[['regular_income_ratios', 'credit_ratios', 'pos_remain_ratios']]
target = test_df['FPF_TARGET']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.83


/Users/jasonzhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/jasonzhou/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


In [238]:
test_df[['FPF_TARGET', 'regular_income_ratios', 'credit_ratios', 'pos_remain_ratios']].corr()

,FPF_TARGET,regular_income_ratios,credit_ratios,pos_remain_ratios
FPF_TARGET,1.000000,-0.004267,0.026909,-0.075893
regular_income_ratios,-0.004267,1.000000,0.167598,0.013484
credit_ratios,0.026909,0.167598,1.000000,-0.081497
pos_remain_ratios,-0.075893,0.013484,-0.081497,1.000000


The highest correlation (in terms of absolute value) with FPF_TARGET is credit_ratios with a correlation of approximately 0.0269. However, it's essential to note that 0.0269 is relatively low, there may a strong linear relationship between the features and 'FPF_TARGET'.